# People Flow Detection using Object Tracking Visualization

---

## 📋 Overview
This project implements a sophisticated system to detect and count people entering and exiting a defined area in a video using object tracking and heatmap visualization. It leverages **YOLOv8** for detection and **ByteTrack** for tracking, complemented by a heatmap to illustrate movement intensity. Bounding boxes are drawn in green with unique track IDs for clear identification.

---

## 🔍 Detection Method
- **Model**: YOLOv8 (pre-trained `yolov8n.pt`) for detecting persons (class 0).
- **Tracker**: ByteTrack ensures consistent tracking of individuals across frames.
- **Libraries**: Utilizes OpenCV, Ultralytics YOLO, Supervision, and Matplotlib for processing and visualization.

---

## 📏 Line Coordinates
- **Upper Line (IN)**: Positioned at `y = frame height / 3`, marked in **blue**.
- **Lower Line (OUT)**: Positioned at `y = 2 * frame height / 3`, marked in **red**.
- Defined using the [PolygonZone](https://polygonzone.roboflow.com/) framework.

---

## 🧮 Logic for IN/OUT Counting
- **IN**: A person is counted as entering when their trajectory moves downward, crossing the upper line (y-coordinate transitions from below `LINE_Y1` to at or above `LINE_Y1`).
- **OUT**: A person is counted as exiting when their trajectory moves upward, crossing the lower line (y-coordinate transitions from above `LINE_Y2` to at or below `LINE_Y2`).
- The system tracks the center y-coordinate of bounding boxes to determine direction, updating counters based on these crossings.

---

## 📤 Output
- **Video**: `output_video.mp4` displays bounding boxes, track IDs, color-coded lines, and live IN/OUT counters.
- **Heatmap**: `heatmap.png` visualizes movement intensity using bounding box center points, normalized with a 'hot' colormap.

---

## 📥 Input & Output Files
- **Input Video**: Paste your video file path here (e.g., `/content/people-walking video.mp4`).
- **Output People Flow Detection Video**: Save as `output_video.mp4`.
- **Heatmap Image**: Save as `heatmap.png`.

---

## 🚀 Setup
1. Install required dependencies in your Colab environment:
   ```bash
   !pip install opencv-python numpy ultralytics supervision matplotlib


# Import Libraries

In [16]:
import cv2
import numpy as np
from ultralytics import YOLO
from supervision import ByteTrack, Color
import supervision as sv
import matplotlib.pyplot as plt

# Load YOLOv8n model

In [17]:
model = YOLO("yolov8n.pt")

# Video input


In [18]:
video_path = "/content/people-walking video.mp4"
cap = cv2.VideoCapture(video_path)


# Get video properties


In [19]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define two horizontal lines


In [20]:
LINE_Y1 = height // 3  # Upper line
LINE_Y2 = 2 * height // 3  # Lower line
LINE_COLOR_UP = Color.BLUE
LINE_COLOR_DOWN = Color.RED

# Initialize counters, tracker, and history


In [21]:
in_count = 0
out_count = 0
tracker = ByteTrack()
track_history = {}
counted_in_ids = set()
counted_out_ids = set()
heatmap = np.zeros((height, width), dtype=np.float32)

# Output video setup


In [22]:
output_path = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Line coordinates for counting


In [23]:
line1 = sv.LineZone(start=sv.Point(0, LINE_Y1), end=sv.Point(width, LINE_Y1))
line2 = sv.LineZone(start=sv.Point(0, LINE_Y2), end=sv.Point(width, LINE_Y2))

# Frame processing


In [24]:
def process_frame(frame, frame_num):
    global in_count, out_count

    results = model(frame, classes=[0])[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    annotated_frame = frame.copy()

    for box, track_id in zip(detections.xyxy, detections.tracker_id):
        x1, y1, x2, y2 = map(int, box)
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Track history
        if track_id not in track_history:
            track_history[track_id] = []
        track_history[track_id].append((center_x, center_y))

        # Count IN/OUT once per person
        if len(track_history[track_id]) > 1:
            prev_x, prev_y = track_history[track_id][-2]
            curr_x, curr_y = track_history[track_id][-1]

            # IN
            if prev_y < LINE_Y1 and curr_y >= LINE_Y1 and track_id not in counted_in_ids:
                in_count += 1
                counted_in_ids.add(track_id)

            # OUT
            elif prev_y > LINE_Y2 and curr_y <= LINE_Y2 and track_id not in counted_out_ids:
                out_count += 1
                counted_out_ids.add(track_id)

        # Draw bounding boxes based on status
        if track_id in counted_in_ids:
            overlay = annotated_frame.copy()
            cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 255, 0), -1)  # Green overlay
            alpha = 0.3
            cv2.addWeighted(overlay, alpha, annotated_frame, 1 - alpha, 0, annotated_frame)
            cv2.putText(annotated_frame, f"ID: {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        elif track_id in counted_out_ids:
            overlay = annotated_frame.copy()
            cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 0, 255), -1)  # Red overlay
            alpha = 0.3
            cv2.addWeighted(overlay, alpha, annotated_frame, 1 - alpha, 0, annotated_frame)
            cv2.putText(annotated_frame, f"ID: {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        else:
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green
            cv2.putText(annotated_frame, f"ID: {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw IN/OUT lines
    cv2.line(annotated_frame, (0, LINE_Y1), (width, LINE_Y1), LINE_COLOR_UP.as_bgr(), 2)
    cv2.line(annotated_frame, (0, LINE_Y2), (width, LINE_Y2), LINE_COLOR_DOWN.as_bgr(), 2)

    # Draw white trajectory lines
    for person_id, points in track_history.items():
        if len(points) > 1:
            for i in range(1, len(points)):
                pt1 = points[i - 1]
                pt2 = points[i]
                cv2.line(annotated_frame, pt1, pt2, (255, 255, 255), 2)

    # Display larger counters
    cv2.putText(annotated_frame, f"IN: {in_count}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 3)
    cv2.putText(annotated_frame, f"OUT: {out_count}", (10, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3)

    return annotated_frame

# Process video


In [25]:
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    annotated_frame = process_frame(frame, frame_num)
    out.write(annotated_frame)
    frame_num += 1


0: 384x640 37 persons, 446.9ms
Speed: 26.7ms preprocess, 446.9ms inference, 44.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 188.7ms
Speed: 5.4ms preprocess, 188.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 146.3ms
Speed: 4.1ms preprocess, 146.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 172.8ms
Speed: 30.0ms preprocess, 172.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 153.2ms
Speed: 6.6ms preprocess, 153.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 173.1ms
Speed: 13.5ms preprocess, 173.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 157.4ms
Speed: 8.5ms preprocess, 157.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 163.3ms
Speed: 4.3ms preprocess, 163.3ms inference, 1.6ms post

# Release video resources


In [26]:
cap.release()
out.release()


# Generate & save improved heatmap


In [27]:
heatmap = np.zeros((height, width), dtype=np.float32)
for track_id in track_history:
    for center_x, center_y in track_history[track_id]:
        if 0 <= center_y < height and 0 <= center_x < width:
            heatmap[int(center_y), int(center_x)] += 1

# Gaussian blur for smoothing


In [28]:
heatmap = cv2.GaussianBlur(heatmap, (21, 21), 10)

# Normalize heatmap
heatmap = np.clip(heatmap, 0, None)
if heatmap.max() > 0:
    heatmap = heatmap / heatmap.max()

# Visualize and save heatmap


In [29]:
plt.figure(figsize=(10, 6))
plt.imshow(heatmap, cmap='hot', interpolation='bilinear', vmin=0, vmax=0.8)
plt.colorbar(label='Intensity')
plt.title('People Movement Heatmap')
plt.axis('off')
plt.savefig('heatmap.png', bbox_inches='tight', pad_inches=0)
plt.close()